In [1]:
import numpy as np
import scipy as scp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numba import jit
from numba import cuda

import os
import glob
import time
import IPython.display

from lat_class_parallel import *
"""
from tqdm.notebook import tqdm
"""

'\nfrom tqdm.notebook import tqdm\n'

In [2]:
kl_sim = lattice_sim()

[2 2 3 3]
initializing catch_tail: [1]
Setting worm to tail: [[2 1]], head: [[[2 1]
  [2 1]]]

with k_delta: [-1]



In [3]:
#seed = 42
#np.random.seed(seed)

In [ ]:
"""l sweep"""
num_iter = 5

#limit the number of iteration/worms
for i in range(num_iter):
    print(f"Iteration {i}")

    print("f before sampling:")
    print(kl_sim.f)
    
    ##sample white sites
    kl_sim.mc_sites(
        kl_sim.w_sites,
        kl_sim.w_target_sites,
        kl_sim.w_link_coord,
        kl_sim.mcmc_w_mod_dim,
        kl_sim.mcmc_w_dl,
        kl_sim.mcmc_w_df,
        kl_sim.mcmc_w_target_df,
        kl_sim.l_links,
        kl_sim.k_links,
        kl_sim.W,
        kl_sim.f,
        kl_sim.n_w_sites,
        kl_sim.move_set,
        kl_sim.w_moves,
        kl_sim.lat_size,
        kl_sim.dim
    )
    
    print("f after sampling:")
    print(kl_sim.f)
    
    ##sample black sites
    kl_sim.mc_sites(
        kl_sim.b_sites,
        kl_sim.b_target_sites,
        kl_sim.b_link_coord,
        kl_sim.mcmc_b_mod_dim,
        kl_sim.mcmc_b_dl,
        kl_sim.mcmc_b_df,
        kl_sim.mcmc_b_target_df,
        kl_sim.l_links,
        kl_sim.k_links,
        kl_sim.W,
        kl_sim.f,
        kl_sim.n_b_sites,
        kl_sim.move_set,
        kl_sim.b_moves,
        kl_sim.lat_size,
        kl_sim.dim
    )
        
    print("f after sampling:")
    print(kl_sim.f)
    
    input("Press Enter to continue...")
    time.sleep(1.0)
    IPython.display.clear_output(wait=True)

Iteration 0
f before sampling:
[[0 0 0]
 [0 0 0]
 [0 0 0]]
MCMC dl:
[-1  1 -1 -1 -1]
f after sampling:
[[-2  0  2]
 [ 2 -2  0]
 [ 0  2 -2]]
MCMC dl:
[-1 -1 -1  1]
f after sampling:
[[-2 -4  2]
 [ 0  2 -2]
 [ 0  4  0]]


In [ ]:
"""main worm loop"""
num_iter = 20

max_iter = 10


finished = False

#limit the number of iteration/worms
for i in range(num_iter):
    print(f"Iteration {i}")
        
    for i_worm in range(kl_sim.num_worms):
        kl_sim.print_env_worm(kl_sim.lat_size, kl_sim.head[i_worm,0], kl_sim.tail[i_worm])
    
    print("before move")
    print("head: ")
    head = kl_sim.head[:,0]
    tail = kl_sim.tail
    print(f"{head}")
    print("tail:")
    print(f"{tail}")
        
    ##randomly choose moves
    """not updating k_change in class!!!"""
    #(move_dim, move_sign, move_i, move_set, moves, head, worm_link_coord, k_delta, dk, df, k_change, f_change, num_worms, lat_size, dim)
    kl_sim.propose_moves(
        kl_sim.move_dim,
        kl_sim.move_sign,
        kl_sim.move_i,
        kl_sim.move_set,
        kl_sim.moves,
        kl_sim.head,
        kl_sim.worm_link_coord,
        kl_sim.k_delta,
        kl_sim.k_links,
        kl_sim.dk, 
        kl_sim.df,
        kl_sim.k_change,
        kl_sim.f_change,
        kl_sim.catch_tail,
        kl_sim.num_worms,
        kl_sim.lat_size,
        kl_sim.dim
    )
        
    ##check collisions!
    #head, hoc_dist_ij, lc_dist_ij, num_worms, np_collisions, collision_worms, non_collision_worms, lc_hoc_bool):
    kl_sim.count_collisions(
        kl_sim.head,
        kl_sim.tail,
        kl_sim.hoc_dist_ij,
        kl_sim.lc_dist_ij,
        kl_sim.num_worms,
        kl_sim.collisions,
        kl_sim.collision_worms,
        kl_sim.non_collision_worms,
        kl_sim.lc_hoc_bool,
        kl_sim.catch_tail
    )
        
    print(f"collisions:\n{kl_sim.collisions}")
    print(f"containing worms: {kl_sim.collision_worms}")
    print(f"not containing worms: {kl_sim.non_collision_worms}")

    print("f before sampling:")
    print(kl_sim.f)
    ##chekc whether there actually are any collisions
    if len(kl_sim.collision_worms) > 0:
        ##yes there are!
        ##sample these first
        ##sample_k_worm_collision_queues(collisions, l_links, k_links, mu, head, tail, worm_link_coord, dk, df, W, f, num_worms)
        kl_sim.sample_k_worm_collision_queues(
            kl_sim.collisions,
            kl_sim.l_links,
            kl_sim.k_links,
            kl_sim.mu,
            kl_sim.head,
            kl_sim.tail,
            kl_sim.worm_link_coord,
            kl_sim.dk,
            kl_sim.df,
            kl_sim.W,
            kl_sim.f,
            kl_sim.worm_closed,
            kl_sim.starting,
            kl_sim.catch_tail,
            kl_sim.num_worms
        )
        ##set the changes of sampled collision worms to zero
        ##for the next step (sample ALL worms including these, their effect will be zero nonetheless
        ##because the change associated with them is 0)
        """DONT UNCOMMENT"""
        kl_sim.set_df_dk_zero(kl_sim.df, kl_dim.dk, kl_sim.collision_worms)
            
    ##sample ALL worms
    ##sample_k_worm_all(l_links, k_links, mu, head, tail, worm_link_coord, i_worm, dk, df, W, f, num_worms)
    ##(l_links, k_links, mu, head, tail, worm_link_coord, dk, df, W, f, worm_closed, num_worms)
    kl_sim.sample_k_worm_all(
        kl_sim.l_links,
        kl_sim.k_links,
        kl_sim.mu,
        kl_sim.head,
        kl_sim.tail,
        kl_sim.worm_link_coord,
        kl_sim.dk,
        kl_sim.df,
        kl_sim.W,
        kl_sim.f,
        kl_sim.worm_closed,
        kl_sim.starting,
        kl_sim.catch_tail,
        kl_sim.num_worms
    )
        
    print("after move")
    print("head: ")
    head = kl_sim.head[:,0]
    tail = kl_sim.tail
    print(f"{head}")
    print("tail:")
    print(f"{tail}")
        
    print("f after sampling:")
    print(kl_sim.f)
    
    input("Press Enter to continue...")
    time.sleep(1.0)
    IPython.display.clear_output(wait=True)
    
    closed_i_worms = [i for i in range(kl_sim.num_worms) if kl_sim.worm_closed[i_worm] == True]
    open_i_worms = [i_worm for i_worm in range(kl_sim.num_worms) if kl_sim.worm_closed[i_worm] == False]
    
    if i < max_iter:
        ##reset finished worms
        for closed_i_worm in closed_i_worms:
            ##head==tail!
            ##reset worm
            kl_sim.reset_worm(kl_sim.tail,
                              kl_sim.head,
                              kl_sim.worm_link_coord,
                              closed_i_worm,
                              kl_sim.k_delta,
                              kl_sim.lat_size,
                              kl_sim.worm_closed,
                              kl_sim.starting,
                              kl_sim.dim
                             )
                
        
    else:   
        ##dont reset worms, but  remove them!
        ##decrease number of worms kl_sim.num_worms
        ##reshape head, tail, dk, df, p
        ##and save the new filtered version
        ##(by removing closed worm indices)
        kl_sim.remove_closed_worms(
            closed_i_worms,
            open_i_worms,
            kl_sim.num_worms,
            kl_sim.head, 
            kl_sim.tail,
            kl_sim.k_delta,
            kl_sim.df0,
            kl_sim.worm_closed,
            kl_sim.starting,
            kl_sim.catch_tail,
            kl_sim.dim
        )
        
        
    
    for i_worm in range(kl_sim.num_worms):
        kl_sim.print_env_worm(kl_sim.lat_size, kl_sim.head[i_worm,0], kl_sim.tail[i_worm])            
            
    input("Press Enter to continue...")
    time.sleep(1.0)
    IPython.display.clear_output(wait=True)

In [ ]:
"""parallelized worm loop"""
"""
num_iter = 20

max_iter = 10


finished = False

#limit the number of iteration/worms
for i in range(num_iter):
    print(f"Iteration {i}")
        
    for i_worm in range(kl_sim.num_worms):
        kl_sim.print_env_worm(kl_sim.lat_size, kl_sim.head[i_worm,0], kl_sim.tail[i_worm])
    
    print("before move")
    print("head: ")
    head = kl_sim.head[:,0]
    tail = kl_sim.tail
    print(f"{head}")
    print("tail:")
    print(f"{tail}")
        
    ##randomly choose moves
    #(move_dim, move_sign, move_i, move_set, moves, head, worm_link_coord, k_delta, dk, df, k_change, f_change, num_worms, lat_size, dim)
    kl_sim.propose_moves(
        kl_sim.move_dim,
        kl_sim.move_sign,
        kl_sim.move_i,
        kl_sim.move_set,
        kl_sim.moves,
        kl_sim.head,
        kl_sim.worm_link_coord,
        kl_sim.k_delta,
        kl_sim.k_links,
        kl_sim.dk, 
        kl_sim.df,
        kl_sim.k_change,
        kl_sim.f_change,
        kl_sim.catch_tail,
        kl_sim.num_worms,
        kl_sim.lat_size,
        kl_sim.dim
    )
        
    ##check collisions!
    #head, hoc_dist_ij, lc_dist_ij, num_worms, np_collisions, collision_worms, non_collision_worms, lc_hoc_bool):
    kl_sim.count_collisions(
        kl_sim.head,
        kl_sim.tail,
        kl_sim.hoc_dist_ij,
        kl_sim.lc_dist_ij,
        kl_sim.num_worms,
        kl_sim.collisions,
        kl_sim.collision_worms,
        kl_sim.non_collision_worms,
        kl_sim.lc_hoc_bool,
        kl_sim.catch_tail
    )
        
    print(f"collisions:\n{kl_sim.collisions}")
    print(f"containing worms: {kl_sim.collision_worms}")
    print(f"not containing worms: {kl_sim.non_collision_worms}")

    print("f before sampling:")
    print(kl_sim.f)
    ##chekc whether there actually are any collisions
    if len(kl_sim.collision_worms) > 0:
        ##yes there are!
        ##sample these first
        ##sample_k_worm_collision_queues(collisions, l_links, k_links, mu, head, tail, worm_link_coord, dk, df, W, f, num_worms)
        kl_sim.sample_k_worm_collision_queues(
            kl_sim.collisions,
            kl_sim.l_links,
            kl_sim.k_links,
            kl_sim.mu,
            kl_sim.head,
            kl_sim.tail,
            kl_sim.worm_link_coord,
            kl_sim.dk,
            kl_sim.df,
            kl_sim.W,
            kl_sim.f,
            kl_sim.worm_closed,
            kl_sim.starting,
            kl_sim.catch_tail,
            kl_sim.num_worms
        )
        ##set the changes of sampled collision worms to zero
        ##for the next step (sample ALL worms including these, their effect will be zero nonetheless
        ##because the change associated with them is 0)
        kl_sim.set_df_dk_zero(kl_sim.df, kl_dim.dk, kl_sim.collision_worms)
            
    ##sample ALL worms
    ##sample_k_worm_all(l_links, k_links, mu, head, tail, worm_link_coord, i_worm, dk, df, W, f, num_worms)
    ##(l_links, k_links, mu, head, tail, worm_link_coord, dk, df, W, f, worm_closed, num_worms)
    kl_sim.sample_k_worm_all(
        kl_sim.l_links,
        kl_sim.k_links,
        kl_sim.mu,
        kl_sim.head,
        kl_sim.tail,
        kl_sim.worm_link_coord,
        kl_sim.dk,
        kl_sim.df,
        kl_sim.W,
        kl_sim.f,
        kl_sim.worm_closed,
        kl_sim.starting,
        kl_sim.catch_tail,
        kl_sim.num_worms
    )
        
    print("after move")
    print("head: ")
    head = kl_sim.head[:,0]
    tail = kl_sim.tail
    print(f"{head}")
    print("tail:")
    print(f"{tail}")
        
    print("f after sampling:")
    print(kl_sim.f)
    
    input("Press Enter to continue...")
    time.sleep(1.0)
    IPython.display.clear_output(wait=True)
    
    closed_i_worms = [i for i in range(kl_sim.num_worms) if kl_sim.worm_closed[i_worm] == True]
    open_i_worms = [i_worm for i_worm in range(kl_sim.num_worms) if kl_sim.worm_closed[i_worm] == False]
    
    if i < max_iter:
        ##reset finished worms
        for closed_i_worm in closed_i_worms:
            ##head==tail!
            ##reset worm
            kl_sim.reset_worm(kl_sim.tail,
                              kl_sim.head,
                              kl_sim.worm_link_coord,
                              closed_i_worm,
                              kl_sim.k_delta,
                              kl_sim.lat_size,
                              kl_sim.worm_closed,
                              kl_sim.starting,
                              kl_sim.dim
                             )
                
        
    else:   
        ##dont reset worms, but  remove them!
        ##decrease number of worms kl_sim.num_worms
        ##reshape head, tail, dk, df, p
        ##and save the new filtered version
        ##(by removing closed worm indices)
        kl_sim.remove_closed_worms(
            closed_i_worms,
            open_i_worms,
            kl_sim.num_worms,
            kl_sim.head, 
            kl_sim.tail,
            kl_sim.k_delta,
            kl_sim.df0,
            kl_sim.worm_closed,
            kl_sim.starting,
            kl_sim.catch_tail,
            kl_sim.dim
        )
        
        
    
    for i_worm in range(kl_sim.num_worms):
        kl_sim.print_env_worm(kl_sim.lat_size, kl_sim.head[i_worm,0], kl_sim.tail[i_worm])            
            
    input("Press Enter to continue...")
    time.sleep(1.0)
    IPython.display.clear_output(wait=True)
"""

In [ ]:
"""
What is missing?
Save df0 when starting worms

combine i_worm loops for more efficiency and parallelization
refrain from exclusive python functionality as much as possible
Set df correctly
Check for tails and accomodate for df0
f update is not working (indexing problem?)
p is not correctly drawn?
if n_iter is exceeded, remove worms and shrink the worm arrays
"""

        ##num_worms
        ##head
        ##tail
        ##k_delta
        ##df0
        
        ##worm_closed
        ##starting
        
        ##reset by counting collisions
        ##collisions
        ##hoc_dist_ij
        ##lc_dist_ij
        
        ##reset by proposing new moves
        ##move_dim
        ##move_sign
        ##move_i
        ##moves
        ##worm_link_coord
        ##dk
        ##df
        
        ##reset by sampling
        """not an attribute?"""
        ##p